I will be working through the notebooks and data from my existing python scripts to plan out the steps required to finish this research.

The first step will be to combine the partial MF datasets from Morningstar. Previously I have regrouped this data into a series of .csv files grouped by country of domicile instead of by fund creation date. I still think this is necessary, as the raw data has been downloaded several times and there is no guarantee that the split between file parts in each folder includes the same funds on either side, while a grouping by domicile will ensure that all funds of a given domicile are in the same file part. This means that only one file part needs to be loaded at a time when processing the data, making it possible to run on moderately sized machines.

In [3]:
# Read mf_info_part-1 into a dataframe in Julia

using DataFrames
using CSV

mf_info = CSV.read("../Data/Raw Data/Mutual Funds/info/mf_info_part-1.csv", DataFrame)
first(mf_info)

Row,FundId,SecId,Domicile,Base Currency,Fund Class Name,Fund Class Legal Name,Fund Standard Name,Fund Legal Name,Firm Name (Asset Manager),Firm Name (Fund Manager),True No-Load,Inception \nDate
,String15?,String15,String31?,String31?,String,String?,String?,String?,String?,String?,String3,String15
1,FS000091C6,F00000NE43,Canada,Canadian Dollar,@rgentum Canadian Equity Portfolio,@RGENTUM CANADIAN EQUITY PORTFOLIO,@rgentum Canadian Equity Portfolio,@rgentum Canadian Equity Portfolio,missing,@rgentum Management & Research Corp.,Yes,2/04/1998


In [6]:
# Read df_local-monthly-gross-returns_part-1 into a dataframe in Julia

using DataFrames
using CSV

df_local_monthly_gross_returns = CSV.read("../Data/Raw Data/Mutual Funds/local-monthly-gross-returns/df_local-monthly-gross-returns_part-1.csv", DataFrame)
first(df_local_monthly_gross_returns)

Row,Name,FundId,SecId,Monthly Gross Return (Local Currency) \n1970-01 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-02 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-03 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-04 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-05 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-06 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-07 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-08 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-09 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-10 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-11 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1970-12 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-01 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-02 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-03 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-04 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-05 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-06 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-07 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-08 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-09 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-10 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-11 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1971-12 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-01 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-02 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-03 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-04 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-05 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-06 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-07 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-08 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-09 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-10 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-11 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1972-12 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-01 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-02 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-03 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-04 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-05 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-06 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-07 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-08 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-09 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-10 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-11 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1973-12 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-01 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-02 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-03 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-04 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-05 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-06 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-07 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-08 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-09 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-10 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-11 \nBase \nCurrency,Monthly Gross Return (Local Currency) \n1974-12 \nBase \nCurrency,Monthly Gross Retu

I've just seen that once again some of the data files are named incorrectly. This time the data files for monthly gross and net returns, both usd and local, begin with df_ instead of mf_.

In [17]:
# Fixing filenames

for folder in ["local-monthly-gross-returns", "local-monthly-net-returns", "usd-monthly-gross-returns", "usd-monthly-net-returns"]
    for file in readdir("../Data/Raw Data/Mutual Funds/$folder")
        if file[1] == 'd'
            mv("../Data/Raw Data/Mutual Funds/$folder/$file", "../Data/Raw Data/Mutual Funds/$folder/mf_"*file[2:end])
        end
    end
end

That didn't work, fixing a new problem now.

In [18]:
# Fixing filenames

for folder in ["local-monthly-gross-returns", "local-monthly-net-returns", "usd-monthly-gross-returns", "usd-monthly-net-returns"]
    for file in readdir("../Data/Raw Data/Mutual Funds/$folder")
        if file[1:7] == "mf_f_f_"
            mv("../Data/Raw Data/Mutual Funds/$folder/$file", "../Data/Raw Data/Mutual Funds/$folder/mf_"*file[8:end])
        end
    end
end

When using VS code it's actually much simpler to inspect data in a .csv preview window rather than loading it into a notebook, so I'll start to do that now.

The first stage, then, involves reading in all parts of the info data so that mapping from fundid to domicile can be made. Then I load each part of every other folder, map each row to its domicile and group each domicile into defined groups, then split up the full dataframe into new .csv parts.